# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set.

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation

    * Il migliore MAE di train

    * Il MAE di test


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni.

In [29]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [30]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training.

In [31]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """

    # Copie dei dati originali
    X_train_copia = X_train.copy()
    X_val_copia = X_val.copy()

       # Standardizzazione prima della PCA (opzionale)
    if hyperparams['use_pca'] and hyperparams['pca_standardize']:
        scaler_pca = StandardScaler()
        X_train_copia = scaler_pca.fit_transform(X_train_copia)
        X_val_copia = scaler_pca.transform(X_val_copia)

    scaler = None
    if hyperparams['use_pca']:
        #implementazione codice PCA
        pca = PCA(n_components= hyperparams['pca_components'])
        X_train_copia = pca.fit_transform(X_train_copia)
        X_val_copia = pca.transform(X_val_copia)

    if hyperparams['data_standardize']:
        # Implementazione codice per la standardizzazione
        scaler = StandardScaler()
        X_train_copia = scaler.fit_transform(X_train_copia)
        X_val_copia = scaler.transform(X_val_copia)

    # Aggiunge il termine costante ai dati
    # ...

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        model = Ridge(alpha=hyperparams['reg_lambda'])
    else:
        model = LinearRegression()

    # addestramento del modello e calcolo predizioni
    model.fit(X_train_copia, y_train)
    y_pred_val = model.predict(X_val_copia)
    y_pred_train = model.predict(X_train_copia)


    # Calcola il MAE per train e validation set
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mae_train = np.mean(np.abs(y_train - y_pred_train))

    return model, mae_val, mae_train, scaler

# Dividi il dataset in training e test set

# tassi per lo split
tasso_addestramento = 0.8
tasso_validazione = 0.25  # 25% di train o 20% del totale
#tasso_test = 0.2

# Concatenare X e y in una matrice unica
dati_completi = np.c_[X, y]

# Calcolare le dimensioni per ciascun set
shape_total = dati_completi.shape[0]
shape_train = int(shape_total * tasso_addestramento)
shape_val = int(shape_total * tasso_validazione)

# 1. Primo split: separiamo il train set dal test set
train_set = dati_completi[:shape_train]
test_set = dati_completi[shape_train:]

# Dividi il training set in training set effettivo e validation set

# 2. Secondo split: separiamo il train set in train effettivo e validation set
val_set = train_set[:shape_val]
train_effettivo_set = train_set[shape_val:]

# Separare le caratteristiche e il target per ogni set
X_train_effettivo, y_train_effettivo = train_effettivo_set[:, :-1], train_effettivo_set[:, -1]
X_val, y_val = val_set[:, :-1], val_set[:, -1]
X_test, y_test = test_set[:, :-1], test_set[:, -1]

#visualizzazione risultati
'''
print("X_train_effettivo:", X_train_effettivo)
print("y_train_effettivo:", y_train_effettivo)
print("X_val:", X_val)
print("y_val:", y_val)
print("X_test:", X_test)
print("y_test:", y_test)
'''

# Trova la configurazione di iperparametri migliore
def find_best_config(X_train_effettivo, y_train_effettivo, X_val, y_val, configs):
    """
    Testa tutte le configurazioni fornite e restituisce la migliore in base alla MAE di validation.
    """

    best_mae_val = float('inf')
    best_model = None
    best_config = None
    best_scaler = None
    best_mae_train = None

    for config in configs:
        try:
            model, mae_train, mae_val, scaler  = pipeline(X_train_effettivo, y_train_effettivo, X_val, y_val, config)
            if mae_val < best_mae_val:
                best_mae_val = mae_val
                best_model = model
                best_config = config
                best_scaler = scaler
                best_mae_train = mae_train
        except Exception as e:
            print(f" Errore con config {config}: {e}")

    return best_model, best_config, best_mae_val, best_mae_train, best_scaler

best_model, best_config, best_mae_val, best_mae_train, best_scaler = find_best_config(X_train_effettivo, y_train_effettivo, X_val, y_val, configs)

# Riallena il modello sul training set completo

def evaluate_on_test(X_train_effettivo, y_train_effettivo, X_val, y_val, X_test, y_test, best_config):
    """
    Riallena il modello sulla configurazione migliore e valuta su test.
    """
    # Unisce train effettivo e validation per creare il set completo di allenamento
    X_train_full = np.concatenate((X_train_effettivo, X_val))
    y_train_full = np.concatenate((y_train_effettivo, y_val))

    # Addestra e valuta il modello sul test set usando la pipeline
    model, _, _, _= pipeline(X_train_full, y_train_full, X_test, y_test, best_config)

    # Predizioni e MAE sul test set
    y_pred_test = model.predict(X_test)
    mae_test = np.mean(np.abs(y_test - y_pred_test))

    return mae_test

mae_test = evaluate_on_test(X_train_effettivo, y_train_effettivo, X_val, y_val, X_test, y_test, best_config)

# Stampa  risultati
print(f'Migliore configurazione: {best_config}')
print(f'Miglior MAE di validation: {best_mae_val}')
print(f'Miglior MAE di train: {best_mae_train}')
print(f'MAE di test: {mae_test}')

 Errore con config {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 0.1}: name 'Ridge' is not defined
 Errore con config {'use_pca': True, 'pca_standardize': True, 'pca_components': 10, 'data_standardize': True, 'use_regularization': False, 'reg_lambda': None}: name 'LinearRegression' is not defined
 Errore con config {'use_pca': True, 'pca_standardize': True, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 0.1}: name 'Ridge' is not defined
 Errore con config {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 1}: name 'Ridge' is not defined
 Errore con config {'use_pca': True, 'pca_standardize': True, 'pca_components': 5, 'data_standardize': False, 'use_regularization': True, 'reg_lambda': 10}: name 'Ridge' is not defined
 Errore con config {'use_pca': True, 'pca_standardize': 

TypeError: 'NoneType' object is not subscriptable